In [3]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import numpy as np
import pandas as pd
import math
import glob

from model import LSTMDriver
from model import HYPERPARAMS

import logging
logging.basicConfig(filename='training.log',level=logging.DEBUG)

In [4]:
PROJECT_ROOT = '../../../..'
LEARNED_DRIVER = 'snakeoil_miner/data'
DIFFICULTY = 'easy'


TRAINING_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, DIFFICULTY, '*.csv']))
TRAINING_DATA = {}
for FILE in TRAINING_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    TRAINING_DATA[FILE] = DF.values

VALIDATION_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, 'validation', '*.csv']))
VALIDATION_DATA = {}
for FILE in VALIDATION_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    VALIDATION_DATA[FILE] = DF.values
    
CUDA = torch.cuda.is_available()
if CUDA:
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor

In [5]:
def save_checkpoint(state, is_best, filepath='latest_checkpoint.pth.tar'):
    torch.save(state, 'checkpoints/' + filepath)
    if is_best:
        torch.save(state, 'checkpoints/best_checkpoint.pth.tar')

In [6]:
def train(training_data, model, criterion, optimzier):
    loss = 0
    for key in training_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden = model.init_hidden()
        optimizer.zero_grad()
        track_sequence = training_data[key]

        targets = track_sequence[:, 0:3]
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets)).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs),  requires_grad=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        track_loss.backward()
        optimizer.step()

        loss += track_loss.data[0]
    return loss

In [7]:
def validate(validation_data, model, criterion):
    loss = 0
    for key in validation_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden = model.init_hidden()
        track_sequence = validation_data[key]

        targets = track_sequence[:, 0:3]
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets), volatile=True).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs), volatile=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        loss += track_loss.data[0]
    return loss

In [8]:
if CUDA:
    model = LSTMDriver(HYPERPARAMS.INPUT_SIZE,
                       HYPERPARAMS.HIDDEN_SIZE,
                       HYPERPARAMS.NUM_LAYERS,
                       HYPERPARAMS.TARGET_SIZE,
                       HYPERPARAMS.BATCH_SIZE).cuda()
    criterion = nn.MSELoss().cuda()
else:
    model = LSTMDriver(HYPERPARAMS.INPUT_SIZE,
                       HYPERPARAMS.HIDDEN_SIZE,
                       HYPERPARAMS.NUM_LAYERS,
                       HYPERPARAMS.TARGET_SIZE,
                       HYPERPARAMS.BATCH_SIZE)
    criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=HYPERPARAMS.LEARNING_RATE)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True)

min_loss = math.inf
losses = {
  'training': [],
  'validation': []
}

for epoch in np.arange(HYPERPARAMS.NUM_EPOCHS):
    if CUDA:
        model.cuda()
        criterion.cuda()
    
    logging.info('Epoch [%d/%d]' %(epoch+1, HYPERPARAMS.NUM_EPOCHS))
    print('Epoch [%d/%d]' %(epoch+1, HYPERPARAMS.NUM_EPOCHS))
    
    is_best = False

    training_loss = train(TRAINING_DATA, model, criterion, optimizer)
    logging.info('--- TRAINING LOSS: %f' % training_loss)
    print('--- TRAINING LOSS: %f' % training_loss)

    validation_loss = validate(VALIDATION_DATA, model, criterion)
    logging.info('--- VALIDATION LOSS: %f' % validation_loss)
    print('--- VALIDATION LOSS: %f' % validation_loss)

    if validation_loss < min_loss:
        logging.info('--- --- best model found so far: %f' % validation_loss)
        print('--- --- best model found so far: %f' % validation_loss)
        min_loss = validation_loss
        is_best = True

    losses['training'].append(training_loss)
    losses['validation'].append(validation_loss)

    model.cpu()
    save_checkpoint({
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'min_loss': min_loss,
          'optimizer' : optimizer.state_dict(),
      }, is_best)

    scheduler.step(validation_loss)
    logging.info('-------------------------------------------------------')
    print('-------------------------------------------------------')

Epoch [1/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.694910
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.340213
--- --- best model found so far: 0.340213
-------------------------------------------------------
Epoch [2/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.272757
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.219879
--- --- best model found so far: 0.219879
-------------------------------------------------------
Epoch [3/150]
--- Parsing track easy-race_01.csv
---

--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.115682
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156462
--- --- best model found so far: 0.156462
-------------------------------------------------------
Epoch [20/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.113950
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.158080
-------------------------------------------------------
Epoch [21/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy

--- TRAINING LOSS: 0.094116
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.159230
-------------------------------------------------------
Epoch [38/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.095797
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.155592
-------------------------------------------------------
Epoch [39/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.097074
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track valida

--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088901
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156671
-------------------------------------------------------
Epoch [57/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088885
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156676
-------------------------------------------------------
Epoch [58/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088868
-

--- VALIDATION LOSS: 0.156688
-------------------------------------------------------
Epoch [75/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088790
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156688
-------------------------------------------------------
Epoch [76/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088790
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156688
-------------------------------------------------------
Epoch [77/150]
--- Parsing track easy-race_01.cs

--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
-------------------------------------------------------
Epoch [94/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088788
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
-------------------------------------------------------
Epoch [95/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088788
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
----------------------

--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
-------------------------------------------------------
Epoch [113/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088788
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
-------------------------------------------------------
Epoch [114/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088788
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALID

--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
-------------------------------------------------------
Epoch [132/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088788
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
-------------------------------------------------------
Epoch [133/150]
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_02.csv
--- Parsing track easy-race_00.csv
--- TRAINING LOSS: 0.088788
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALID

--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.156690
-------------------------------------------------------
